In [181]:
import os
import json

import pandas as pd
BASE_DIR = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/"

In [182]:
from collections.abc import MutableMapping

def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str ='.') -> MutableMapping:
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            if type(v) == list:
                v = str(v)
            items.append((new_key, v))
    return dict(items)

In [183]:
from ray.rllib.agents.ddpg.td3 import TD3Trainer
default_tde_config = TD3Trainer.get_default_config()
default_tde_config = flatten_dict(default_tde_config)

from ray.rllib.agents.ppo.ppo import PPOTrainer
default_ppo_config = PPOTrainer.get_default_config()
default_ppo_config = flatten_dict(default_ppo_config)

from ray.rllib.agents.sac.sac import SACTrainer
default_sac_config = SACTrainer.get_default_config()
default_sac_config = flatten_dict(default_sac_config)

from ray.rllib.agents.a3c.a3c import A3CTrainer
default_a3c_config = A3CTrainer.get_default_config()
default_a3c_config = flatten_dict(default_a3c_config)

In [184]:
default_tde_config

{'num_workers': 0,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 1,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 0.0001,
 'train_batch_size': 100,
 'model._use_default_native_models': False,
 'model._disable_preprocessor_api': False,
 'model.fcnet_hiddens': '[256, 256]',
 'model.fcnet_activation': 'tanh',
 'model.conv_filters': None,
 'model.conv_activation': 'relu',
 'model.post_fcnet_hiddens': '[]',
 'model.post_fcnet_activation': 'relu',
 'model.free_log_std': False,
 'model.no_final_linear': False,
 'model.vf_share_layers': True,
 'model.use_lstm': False,
 'model.max_seq_len': 20,
 'model.lstm_cell_size': 256,
 'model.lstm_use_prev_action': False,
 'model.lstm_use_prev_reward': False,
 'model._time_major': False,
 'model.use_attention': False,
 'model.attention_num_transformer_units': 1,
 'model.attention_dim': 64,
 'model.attention_num_heads': 1,
 'model.attention_head_dim': 32,
 'model.attention_memory_inference': 50,
 'model

In [187]:
table = []
for (root, dirs, files) in os.walk(BASE_DIR):
    if "params.json" not in files:
        continue
    
    experimentName = root.replace(BASE_DIR, "")
    with open(root+"/params.json", "r") as f:
        params = json.load(f)
    newDict = flatten_dict(params)
    experimentName = experimentName.split("/")
    newDict["algo"] = experimentName[0]
    if len (experimentName) == 3:
        newDict["experiment"] = experimentName[1]
        newDict["run"] = experimentName[2]
    else:
        newDict["experiment"] = experimentName[1] + "/" + experimentName[2]
        newDict["run"] = experimentName[3]
    table.append(newDict)

In [189]:
for exp in table:
    if exp["algo"]=="TD3":
        for k,v in default_tde_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="PPO":
        for k,v in default_ppo_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="A3C":
        for k,v in default_a3c_config.items():
            if k not in exp:
                exp[k] = v
    elif exp["algo"]=="SAC":
        for k,v in default_sac_config.items():
            if k not in exp:
                exp[k] = v
    else:
        continue
        raise ValueError("No default config for algo: {}".format(exp["algo"]))

In [190]:
df = pd.DataFrame(table)
df.shape

(66, 200)

In [191]:
df = df[df["algo"]=="TD3"]

In [192]:
df.fillna("-", inplace=True)

In [193]:
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)
df.shape

(47, 22)

In [194]:
#df = df.loc[:, ["algo", "experiment", "run"] + sorted([c for c in df.columns if c not in ["algo", "experiment", "run"]])]
#df = df.sort_values(["algo", "experiment"])
df = df.loc[:, ["experiment", "run"] + sorted([c for c in df.columns if c not in ["algo", "experiment", "run"]])]
df = df.sort_values(["experiment"])

In [195]:
df.to_csv("ExperimentsTable_obst.csv", sep=";", encoding="cp1251", index=False)